In [11]:
from agents.agent import Agent
from load_models import ModelLoader
from prompt_processing import PromptProcessing
from tools.tools import ToolExecutor, ToolRegistry
from tools.tools import CalculatorTool, EquationSolverTool, SymbolicMathTool, MatrixToolAgent, StatisticsToolAgent

registry = ToolRegistry()
registry.register(EquationSolverTool())
registry.register(SymbolicMathTool())
registry.register(MatrixToolAgent())
registry.register(StatisticsToolAgent())

executor = ToolExecutor(registry=registry)


In [5]:
question = "Find x, y: 2x+y=1, 3x+2y=3"

p_proc = PromptProcessing()

p_proc.load(filename="system_prompts.json")

SYSTEM_PROMPT = p_proc.get_system_prompt()

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": question}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

outputs = model.generate(
    **tokenizer(prompt, return_tensors="pt").to(model.device),
    max_new_tokens=200,
    do_sample=False,   
    temperature=0.0   
)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
tool_call = p_proc.extract_assistant_json(text)




In [12]:
result = executor.execute(tool_call)
result

NameError: name 'tool_call' is not defined

In [8]:
tool_call

{'tool': 'calculator',
 'action': None,
 'args': {'expression': '2*x + y = 1, 3*x + 2*y = 3'}}

In [11]:
SYSTEM_PROMPT

'You are a precise math tool selector agent. Your ONLY task is to analyze the user query and output exactly one tool call in valid JSON format.\n\nSTRICT RULES (NO EXCEPTIONS):\n- Output ONLY a single valid JSON object.\n- No explanations, no reasoning, no markdown, no extra text whatsoever.\n- Never perform calculations yourself.\n- Use tool and action names EXACTLY as listed below.\n- For \'calculator\' tool, set "action": null.\n- For all other tools, "action" must be a string matching one of the listed actions.\n\nALLOWED TOOLS:\n\n1. calculator\n   - Description: Evaluate pure numeric expressions (no variables).\n   - Action: null\n   - Args: {"expression": string}\n\n2. equation_solver\n   - Description: Solve equations or systems.\n   - Actions: "solve_equation", "solve_system"\n   - Args (solve_equation): {"equation": string, "var": string}\n   - Args (solve_system): {"equations": list[string], "vars": list[string]}\n\n3. symbolic_math\n   - Description: Symbolic operations.\n 

ChatCompletionMessage(content='The solution to the system of equations is correct:\n\n\\[\n\\begin{aligned}\nx + y &= 1 \\\\\n2x + 4y &= 5\n\\end{aligned}\n\\]\n\nSolving gives:\n\n\\[\nx = -\\frac{1}{2},\\quad y = \\frac{3}{2}\n\\]\n\nIf you obtained a different result, please double-check your steps. I am confident in this solution.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning='Alright — the user just asked me to check the solution to a system of equations:  \n\\( x + y = 1 \\) and \\( 2x + 4y = 5 \\). They followed up with "Are you sure? Think carefully."\n\nFirst, I need to verify the calculations step-by-step to confirm accuracy.\n\nStarting with the first equation:  \n\\( x + y = 1 \\).  \nIf I solve for \\( x \\), I get \\( x = 1 - y \\).\n\nSubstitute into the second equation:  \n\\( 2(1 - y) + 4y = 5 \\)  \nSimplify: \\( 2 - 2y + 4y = 5 \\) → \\( 2 + 2y = 5 \\)  \nThen \\( 2y = 3 \\) → \\( y = 1.5 \\).\n\nNow pl

In [4]:
SYSTEM_PROMPT

'You are a precise math tool selector agent. Your ONLY task is to analyze the user query and output exactly one tool call in valid JSON format.\n\nSTRICT RULES (NO EXCEPTIONS):\n- Output ONLY a single valid JSON object.\n- No explanations, no reasoning, no markdown, no extra text whatsoever.\n- Never perform calculations yourself.\n- Use tool and action names EXACTLY as listed below.\n- For \'calculator\' tool, set "action": null.\n- For all other tools, "action" must be a string matching one of the listed actions.\n\nALLOWED TOOLS:\n\n1. calculator\n   - Description: Evaluate pure numeric expressions (no variables).\n   - Action: null\n   - Args: {"expression": string}\n\n2. equation_solver\n   - Description: Solve equations or systems.\n   - Actions: "solve_equation", "solve_system"\n   - Args (solve_equation): {"equation": string, "var": string}\n   - Args (solve_system): {"equations": list[string], "vars": list[string]}\n\n3. symbolic_math\n   - Description: Symbolic operations.\n 

In [2]:
from prompt_processing import PromptProcessing
from agents.agent import Agent

import os

OPEN_ROUTER_API = os.getenv("OPEN_ROUTER_API")

p_proc = PromptProcessing()
p_proc.load("system_prompts.json")
SYSTEM_PROMPT = p_proc.get_system_prompt()

agent = Agent(SYSTEM_PROMPT=SYSTEM_PROMPT, p_proc=p_proc, executor=executor, OPEN_ROUTER_API=OPEN_ROUTER_API)

question = "What is the x and y for this equations: x+y=1, 2x+4y=5"

response = agent.run_agent(question=question)

print(response)

ChatCompletionMessage(content='{\n  "tool": "equation_solver",\n  "action": "solve_system",\n  "args": {\n    "equations": ["x+y=1", "2x+4y=5"],\n    "vars": ["x", "y"]\n  }\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning='First, the user query is: "What is the x and y for this equations: x+y=1, 2x+4y=5". This is asking for the solution to a system of linear equations.\n\nI need to analyze this and output exactly one tool call in valid JSON format. No extra text, just JSON.\n\nLooking at the allowed tools:\n\n- calculator: for pure numeric expressions.\n\n- equation_solver: for solving equations or systems. Actions are "solve_equation" or "solve_system".\n\n- symbolic_math: for symbolic operations like derivative, integral, simplify.\n\n- matrix: for matrix operations.\n\n- statistics: for statistical calculations.\n\nThis is a system of two linear equations in two variables, x and y. So, it\'s a system of equations, not

In [3]:
type(response)

openai.types.chat.chat_completion_message.ChatCompletionMessage

In [8]:
import json
content = json.loads(response.content)

content

{'tool': 'equation_solver',
 'action': 'solve_system',
 'args': {'equations': ['x+y=1', '2x+4y=5'], 'vars': ['x', 'y']}}

In [10]:
result = executor.execute(content)
result

ValueError: Error solving system: invalid syntax (<string>, line 1)

In [1]:
from agents.crew_run import run

a = run()
a

d:\Forum\Forum\AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 290/290 [00:00<00:00, 581.86it/s, Materializing param=model.norm.weight]                              
d:\Forum\Forum\AI\venv\Lib\site-packages\peft\peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self

ValidationError: 5 validation errors for Agent
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<tools.tools.CalculatorTo...t at 0x0000016DD5407F20>, input_type=CalculatorTool]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
tools.1
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<tools.tools.EquationSolv...t at 0x0000016E3BAC5370>, input_type=EquationSolverTool]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
tools.2
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<tools.tools.SymbolicMath...t at 0x0000016E3BFFDF40>, input_type=SymbolicMathTool]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
tools.3
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<tools.tools.MatrixToolAg...t at 0x0000016E3C02D0D0>, input_type=Mat

In [2]:
from agents.multi_agents import MathCrew

a = MathCrew().get_tasks_config()
a

ValidationError: 5 validation errors for Agent
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<tools.tools.CalculatorTo...t at 0x0000016DD5407F20>, input_type=CalculatorTool]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
tools.1
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<tools.tools.EquationSolv...t at 0x0000016E3BAC5370>, input_type=EquationSolverTool]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
tools.2
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<tools.tools.SymbolicMath...t at 0x0000016E3BFFDF40>, input_type=SymbolicMathTool]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
tools.3
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<tools.tools.MatrixToolAg...t at 0x0000016E3C02D0D0>, input_type=Mat